<a href="https://colab.research.google.com/github/Pongpang-2102/Text_Mining_and_Sentiment_Analytics_KDAI_projects/blob/main/KDAI_Exercise_practice_Topic_Modeling_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pythainlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 87.8 MB/s eta 0:00:00


In [ ]:
!pip install --no-cache-dir ipysheet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 316.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from pythainlp.tokenize import word_tokenize, Tokenizer
from pythainlp.corpus.common import thai_words

from difflib import SequenceMatcher, Differ
from pprint import pprint

import re
import sqlite3

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from pythainlp.corpus import thai_stopwords
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import ipywidgets as widgets
from ipysheet import from_dataframe, to_dataframe

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Lab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

Mounted at /content/drive


In [ ]:
pwd

'/content'

In [ ]:
os.chdir('drive/MyDrive/Classroom/05177107 - Text Mining And Sentiment Analytics/20231001 #11 Topic Modelling/HW')

In [ ]:
pwd

'/content/drive/MyDrive/Classroom/05177107 - Text Mining And Sentiment Analytics/20231001 #11 Topic Modelling/HW'

In [ ]:
ls

'Copy of Topic_Modeling_News.ipynb'   Topic_Modeling_News.ipynb
 input/                              'ผลส่งอาจารย์ (Output)'/
 output/


In [ ]:
# Input File
SOURCE_File = 'input/news_0965-0866.csv'
THAI_CUSTOM_WORDS = 'input/thai_custom_words.txt'
THAI_STOP_WORDS = 'input/thai_stop_words.txt'
THAI_CORRECT_WORDS = 'input/thai_correct_words.txt'

In [ ]:
#os.listdir(SOURCE_File)

In [ ]:
# Output File
DICTIONARY = 'output/dictionary.txt'
TOPIC_NAME = 'output/topic_name.csv'
TOPIC = 'output/topic.csv'

In [ ]:
outputall = 'output/'
outputall

'output/'

In [ ]:
!rm outputall/*

!ls -l outputall/*
#for windows
#!dir .\output

rm: cannot remove 'outputall/*': No such file or directory
ls: cannot access 'outputall/*': No such file or directory


# Parameter

In [ ]:
TOPIC_NUMBER = 5

In [ ]:
PREDICT_THRESHOLD = 0.001

In [ ]:
df = pd.read_csv(SOURCE_File, usecols=[0,1,2] ,names=['id', 'topic','sentence'], header=0)

In [ ]:
df.head(10)

,id,topic,sentence
0,1,ข่าวสังคม,ประกันสังคมยะลา ขอผู้ประกันตนมาตรา 40 ที่ยังไม...
1,2,ข่าวสังคม,นพค. 56 อุบลราชธานี ให้การช่วยเหลือ รื้อถอน ตั...
2,3,ข่าวสังคม,กาชาดยะลา มอบถุงยังชีพสนับสนุนการขับเคลื่อนการ...
3,4,ข่าวสังคม,"กิจกรรม ""โครงการลดความเหลื่อมล้ำทางการศึกษา""เท..."
4,5,ข่าวสังคม,ประชาชนจังหวัดอำนาจเจริญ สามารถลงทะเบียนบัตร?ส...
5,6,ข่าวสังคม,จัดจัดงาน 100 ปี ยุวกาชาดไทย วันคล้ายวันสถาปนา...
6,7,ข่าวสังคม,กฟผ. เหมืองแม่เมาะ ซ้อมแผนฉุกเฉินระงับอัคคีภัย...
7,8,ข่าวสังคม,รัฐบาล รายงานผู้ป่วยโควิด 19 พบหมอออนไลน์และรั...
8,9,ข่าวสังคม,ยะลา ประชาชน ทยอย จับจ่าย “คนละครึ่งเฟส 5” วันแรก
9,10,ข่าวสังคม,สมาชิกวุฒิสภาลงพื้นที่พบประชาชนในพื้นที่จังหวั...


In [ ]:
import duckdb

In [ ]:
Query = 'select distinct(topic) from df'
#Query = 'select * from df'

In [ ]:
qdf = duckdb.query(Query).to_df()
qdf

,topic
0,ข่าวสังคม
1,ข่าวเศรษฐกิจ ท่องเที่ยว
2,ข่าวสาธารณภัย


In [ ]:
opt = qdf['topic'].to_list()

In [ ]:
menu = widgets.Dropdown(
    options=opt,
    value='ข่าวสังคม',
    description='topic:')

In [ ]:
menu

Dropdown(description='topic:', options=('ข่าวสังคม', 'ข่าวเศรษฐกิจ ท่องเที่ยว', 'ข่าวสาธารณภัย'), value='ข่าวส…

In [ ]:
sql = "select id, sentence from df where topic='%s'" % (menu.value)
print(sql)
qdf = duckdb.query(sql).to_df()
#qdf =  pandasql.sqldf("select id, sentence from df where topic='%s'" % (menu.value))

qdf

select id, sentence from df where topic='ข่าวสังคม'


,id,sentence
0,1,ประกันสังคมยะลา ขอผู้ประกันตนมาตรา 40 ที่ยังไม...
1,2,นพค. 56 อุบลราชธานี ให้การช่วยเหลือ รื้อถอน ตั...
2,3,กาชาดยะลา มอบถุงยังชีพสนับสนุนการขับเคลื่อนการ...
3,4,"กิจกรรม ""โครงการลดความเหลื่อมล้ำทางการศึกษา""เท..."
4,5,ประชาชนจังหวัดอำนาจเจริญ สามารถลงทะเบียนบัตร?ส...
...,...,...
54774,54775,บริษัท NT นำโครงการเพาะพันธุ์ดี NT Youth Club ...
54775,54776,ชมรมแม่บ้านมหาดไทยจังหวัดอำนาจเจริญ รถเข็นนั่ง...
54776,54777,เหล่ากาชาดอำนาจเจริญมอบบ้านให้ผู้ด้อยโอกาส ผู้...
54777,54778,ชมรมแม่บ้านมหาดไทยจังหวัดอำนาจเจริญ มอบบ้านตาม...


In [ ]:
qdf.shape

(54779, 2)

# EDA

In [ ]:
custom_words_list = set(thai_words())
len(custom_words_list)

62068

In [ ]:
with open(THAI_CUSTOM_WORDS, encoding='utf8') as f:
    words = [line.strip() for line in f]

words

['นพค.',
 'พ.ศ.',
 'สปสช.',
 'จ.',
 'ตง.',
 'พมจ.',
 'รพ.',
 'พท.',
 'กศน.',
 'รมว.',
 'ศึกษาฯ',
 'ศูนย์บริการ',
 'IMT-GT',
 'Kick Off',
 'นครศรีธรรมราช',
 'ไม้ไผ่',
 'ชาวนา',
 'เมืองยาง',
 'รับเสด็จฯ',
 'เดชอุดม',
 'เทศบาล',
 'นครยะลา',
 'ยุวกาชาด',
 'บุคคลากร',
 'แม่เมาะ',
 'โควิด19',
 'ขอนคลาน',
 'ควนนกหว้า',
 'สายตง',
 'ขยาย',
 'ความกว้าง',
 'สำนักวิเคราะห์',
 'หาดปากเมง',
 'สายตง',
 'ตาโกน',
 'การลงทะเบียน',
 'มหามงคล',
 'สมเด็จพระบรมราชชนนีพันปีหลวง',
 'พื้นฟู',
 'การรถไฟฯ',
 'พระบรมราชินูปถัมภ์',
 'จิตอาสา',
 'ผู้ประกอบการ',
 'การดำเนินงาน',
 'ชาวนา',
 'ลำมูล',
 'สโตย',
 'เพาะเลี้ยง',
 'สัตว์น้ำ',
 'การอาชีวศึกษา',
 'สรรพยา',
 'เลิศรัฐ',
 'นครหาดใหญ่',
 'แม่เมาะ',
 'คอนเซ็ป',
 'น้ำพอง',
 'ผู้ตรวจราชการ',
 'พระราชกุศล',
 'เต่างอย',
 'จิตอาสา',
 'อ่างศิลา',
 'ปศุวัตว์',
 'การบริหาร',
 'งานบุคคล',
 'กศจ.',
 'อ.ก.ค.ศ.',
 'โอมิครอน',
 'ผู้ร้องเรียน',
 'สีชัง',
 'บางละมุง',
 'มุจลินท์นาคราช',
 'องค์พระ',
 'ท่าปอมคลองสองน้ำ',
 'เขาคราม',
 'ชะอำ',
 'ศรีสวรินทิรา',
 'มทส.',
 'ธรรมวิทยา',

In [ ]:
custom_words_list.update(words)
len(custom_words_list)

62342

In [ ]:
text = "โอเคบ่พวกเรารักภาษาบ้านเกิด"

custom_tokenizer = Tokenizer(custom_words_list)
custom_tokenizer.word_tokenize(text)

['โอเค', 'บ่', 'พวกเรา', 'รัก', 'ภาษา', 'บ้านเกิด']

In [ ]:
pun = '"#\'()*,;<=>[\\]^_`{|}~/?!“”'
pun

'"#\'()*,;<=>[\\]^_`{|}~/?!“”'

In [ ]:
qdf['clean_documents'] = qdf['sentence'].str.replace(r'[%s]' % (pun), '', regex=True).apply(lambda x: custom_tokenizer.word_tokenize(x))

In [ ]:
qdf.head()

,id,sentence,clean_documents
0,1,ประกันสังคมยะลา ขอผู้ประกันตนมาตรา 40 ที่ยังไม...,"[ประกันสังคม, ยะลา, , ขอ, ผู้ประกันตน, มาตรา,..."
1,2,นพค. 56 อุบลราชธานี ให้การช่วยเหลือ รื้อถอน ตั...,"[นพค., , 56, , อุบลราชธานี, , ให้การ, ช่วยเ..."
2,3,กาชาดยะลา มอบถุงยังชีพสนับสนุนการขับเคลื่อนการ...,"[กาชาด, ยะลา, , มอบ, ถุง, ยังชีพ, สนับสนุน, ก..."
3,4,"กิจกรรม ""โครงการลดความเหลื่อมล้ำทางการศึกษา""เท...","[กิจกรรม, , โครงการ, ลด, ความเหลื่อมล้ำ, ทาง,..."
4,5,ประชาชนจังหวัดอำนาจเจริญ สามารถลงทะเบียนบัตร?ส...,"[ประชาชน, จังหวัด, อำนาจเจริญ, , สามารถ, ลงทะ..."


In [ ]:
detokenized_doc = []
for i in range(len(qdf)):
    t = ' '.join(map(str, qdf['clean_documents'][i]))
    detokenized_doc.append(t)
qdf['clean_documents'] = detokenized_doc

In [ ]:
qdf.head()

,id,sentence,clean_documents
0,1,ประกันสังคมยะลา ขอผู้ประกันตนมาตรา 40 ที่ยังไม...,ประกันสังคม ยะลา ขอ ผู้ประกันตน มาตรา 40 ...
1,2,นพค. 56 อุบลราชธานี ให้การช่วยเหลือ รื้อถอน ตั...,นพค. 56 อุบลราชธานี ให้การ ช่วยเหลือ ร...
2,3,กาชาดยะลา มอบถุงยังชีพสนับสนุนการขับเคลื่อนการ...,กาชาด ยะลา มอบ ถุง ยังชีพ สนับสนุน การ ขับเค...
3,4,"กิจกรรม ""โครงการลดความเหลื่อมล้ำทางการศึกษา""เท...",กิจกรรม โครงการ ลด ความเหลื่อมล้ำ ทาง การศึก...
4,5,ประชาชนจังหวัดอำนาจเจริญ สามารถลงทะเบียนบัตร?ส...,ประชาชน จังหวัด อำนาจเจริญ สามารถ ลงทะเบียน ...


In [ ]:
qdf['clean_documents'] = qdf['clean_documents'].fillna('').apply(lambda x: x.lower())

In [ ]:
# tokenization
tokenized_doc = qdf['clean_documents'].fillna('').apply(lambda x: x.split())

In [ ]:
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [ ]:
stopwords = thai_stopwords()
len(stopwords)

1030

In [ ]:
with open(THAI_STOP_WORDS) as f:
    new_stopwords_list = [line.strip() for line in f]

new_stopwords_list

['ๆ', '.', 'อย่า', 'ไม่', 'พระ', '']

In [ ]:
stopwords = stopwords.union(new_stopwords_list)
len(stopwords)

1034

In [ ]:
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stopwords])

In [ ]:
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if len(item) > 1])

In [ ]:
with open(THAI_CORRECT_WORDS) as f:
    thai_correct_word_list = [line.strip() for line in f]

# thai_correct_word_list

In [ ]:
correct_pair={}

for pair in thai_correct_word_list:
    correct_pair[pair.split()[0].strip()]=pair.split()[1].strip()

In [ ]:
doc = []
for tokenized_list in tokenized_doc:
    for index, word in enumerate(tokenized_list):
        if word in correct_pair:
            tokenized_list[index]=correct_pair[word]
    doc.append(tokenized_list)

tokenized_doc = pd.Series(doc)

In [ ]:
# de-tokenization
detokenized_doc = []
for i in range(len(qdf)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
qdf['clean_documents'] = detokenized_doc

In [ ]:
qdf

,id,sentence,clean_documents
0,1,ประกันสังคมยะลา ขอผู้ประกันตนมาตรา 40 ที่ยังไม...,ประกันสังคม ยะลา ผู้ประกันตน มาตรา 40 เงินสมทบ...
1,2,นพค. 56 อุบลราชธานี ให้การช่วยเหลือ รื้อถอน ตั...,หน่วยพัฒนาการเคลื่อนที่ 56 อุบลราชธานี ให้การ ...
2,3,กาชาดยะลา มอบถุงยังชีพสนับสนุนการขับเคลื่อนการ...,กาชาด ยะลา มอบ ถุง ยังชีพ สนับสนุน ขับเคลื่อน ...
3,4,"กิจกรรม ""โครงการลดความเหลื่อมล้ำทางการศึกษา""เท...",กิจกรรม โครงการ ลด ความเหลื่อมล้ำ การศึกษา เทศ...
4,5,ประชาชนจังหวัดอำนาจเจริญ สามารถลงทะเบียนบัตร?ส...,ประชาชน จังหวัด อำนาจเจริญ ลงทะเบียน บัตร สวัส...
...,...,...,...
54774,54775,บริษัท NT นำโครงการเพาะพันธุ์ดี NT Youth Club ...,บริษัท nt โครงการ เพาะพันธุ์ ดี nt youth club ...
54775,54776,ชมรมแม่บ้านมหาดไทยจังหวัดอำนาจเจริญ รถเข็นนั่ง...,ชมรม แม่บ้าน มหาดไทย จังหวัด อำนาจเจริญ รถเข็น...
54776,54777,เหล่ากาชาดอำนาจเจริญมอบบ้านให้ผู้ด้อยโอกาส ผู้...,กาชาด อำนาจเจริญ มอบ บ้าน ผู้ด้อยโอกาส ผู้ยากไ...
54777,54778,ชมรมแม่บ้านมหาดไทยจังหวัดอำนาจเจริญ มอบบ้านตาม...,ชมรม แม่บ้าน มหาดไทย จังหวัด อำนาจเจริญ มอบ บ้...


In [ ]:
vectorizer = TfidfVectorizer(smooth_idf=True, tokenizer=lambda x: x.split())
X = vectorizer.fit_transform(qdf['clean_documents'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
dictionary = vectorizer.get_feature_names_out() # back of words = 19717

In [ ]:
with open(DICTIONARY, 'w') as f:
    f.write('\n'.join(dictionary))

In [ ]:
X.toarray().shape

(54779, 19717)

# Topic Modeling

In [ ]:
svd_model = TruncatedSVD(n_components=TOPIC_NUMBER, algorithm='randomized', n_iter=5000, random_state=99)
lsa = svd_model.fit_transform(X) # Latent from 19717 to 5

In [ ]:
lsa.shape

(54779, 5)

In [ ]:
topic_columns = ['topic_'+str(i+1) for i in range(TOPIC_NUMBER)]
topic_columns

['topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5']

In [ ]:
pd.options.display.float_format = '{:,.3f}'.format

topic_encoded_df = pd.DataFrame(lsa, columns = topic_columns)
topic_encoded_df["documents"] = qdf['clean_documents']
topic_encoded_df["sentence"] = qdf['sentence']
# display(topic_encoded_df[["sentence", "documents"]+topic_columns])

In [ ]:
topic_encoded_df.head()

,topic_1,topic_2,topic_3,topic_4,topic_5,documents,sentence
0,0.019,-0.010,0.003,-0.009,0.005,ประกันสังคม ยะลา ผู้ประกันตน มาตรา 40 เงินสมทบ...,ประกันสังคมยะลา ขอผู้ประกันตนมาตรา 40 ที่ยังไม...
1,0.078,-0.047,-0.000,-0.057,0.033,หน่วยพัฒนาการเคลื่อนที่ 56 อุบลราชธานี ให้การ ...,นพค. 56 อุบลราชธานี ให้การช่วยเหลือ รื้อถอน ตั...
2,0.123,-0.068,0.003,-0.116,0.009,กาชาด ยะลา มอบ ถุง ยังชีพ สนับสนุน ขับเคลื่อน ...,กาชาดยะลา มอบถุงยังชีพสนับสนุนการขับเคลื่อนการ...
3,0.079,-0.051,0.010,0.044,0.022,กิจกรรม โครงการ ลด ความเหลื่อมล้ำ การศึกษา เทศ...,"กิจกรรม ""โครงการลดความเหลื่อมล้ำทางการศึกษา""เท..."
4,0.091,-0.046,0.011,-0.045,0.066,ประชาชน จังหวัด อำนาจเจริญ ลงทะเบียน บัตร สวัส...,ประชาชนจังหวัดอำนาจเจริญ สามารถลงทะเบียนบัตร?ส...


In [ ]:
encoding_matrix = pd.DataFrame(svd_model.components_, index = topic_columns, columns = (dictionary)).T
encoding_matrix

,topic_1,topic_2,topic_3,topic_4,topic_5
%...,0.000,-0.000,-0.000,-0.000,-0.000
--,0.000,-0.000,-0.000,0.000,-0.000
-coffee,0.000,-0.000,0.000,-0.000,0.000
-ed,0.000,-0.000,-0.000,-0.000,-0.000
-gistda,0.000,-0.000,0.000,-0.000,0.000
...,...,...,...,...,...
‘กัมปง,0.000,-0.000,0.000,-0.000,0.000
’มจ,0.000,0.000,-0.000,-0.000,0.000
’ว,0.000,-0.000,-0.000,-0.000,-0.000
’’,0.000,0.000,-0.001,-0.000,0.001


In [ ]:
pd.DataFrame(svd_model.components_, index = topic_columns, columns = (dictionary))

,%...,--,-coffee,-ed,-gistda,-ordinator,-phuketthailand,..,...,....,...,๘๐,๙๐,๙๐๔,๙๘,๙๙,‘กัมปง,’มจ,’ว,’’,….
topic_1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.004,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
topic_2,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.002,0.001,-0.000,...,-0.000,0.000,0.000,0.000,0.001,-0.000,0.000,-0.000,0.000,-0.000
topic_3,-0.000,-0.000,0.000,-0.000,0.000,0.000,0.000,0.001,0.005,0.000,...,0.000,-0.000,-0.000,-0.000,0.001,0.000,-0.000,-0.000,-0.001,0.000
topic_4,-0.000,0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.002,-0.002,-0.000,...,-0.000,-0.000,-0.000,0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000
topic_5,-0.000,-0.000,0.000,-0.000,0.000,0.000,0.000,0.002,0.005,0.001,...,0.000,-0.000,-0.000,0.000,0.000,0.000,0.000,-0.000,0.001,-0.000


In [ ]:
encoding_matrix.head()

,topic_1,topic_2,topic_3,topic_4,topic_5
%...,0.000,-0.000,-0.000,-0.000,-0.000
--,0.000,-0.000,-0.000,0.000,-0.000
-coffee,0.000,-0.000,0.000,-0.000,0.000
-ed,0.000,-0.000,-0.000,-0.000,-0.000
-gistda,0.000,-0.000,0.000,-0.000,0.000


In [ ]:
encoding_matrix = encoding_matrix.rename_axis('words')

In [ ]:
encoding_matrix.head()

,topic_1,topic_2,topic_3,topic_4,topic_5
words,,,,,
%...,0.000,-0.000,-0.000,-0.000,-0.000
--,0.000,-0.000,-0.000,0.000,-0.000
-coffee,0.000,-0.000,0.000,-0.000,0.000
-ed,0.000,-0.000,-0.000,-0.000,-0.000
-gistda,0.000,-0.000,0.000,-0.000,0.000


In [ ]:
encoding_matrix.index

Index(['%...', '--', '-coffee', '-ed', '-gistda', '-ordinator',
       '-phuketthailand', '..', '...', '....',
       ...
       '๘๐', '๙๐', '๙๐๔', '๙๘', '๙๙', '‘กัมปง', '’มจ', '’ว', '’’', '….'],
      dtype='object', name='words', length=19717)

สร้าง column words ขึ้นมาใช้ query (ปกติ มีแต่ index words แสดงข้างหน้าเท่านั้น)

In [ ]:
encoding_matrix['words'] = encoding_matrix.index

In [ ]:
encoding_matrix.head()

,topic_1,topic_2,topic_3,topic_4,topic_5,words
words,,,,,,
%...,0.000,-0.000,-0.000,-0.000,-0.000,%...
--,0.000,-0.000,-0.000,0.000,-0.000,--
-coffee,0.000,-0.000,0.000,-0.000,0.000,-coffee
-ed,0.000,-0.000,-0.000,-0.000,-0.000,-ed
-gistda,0.000,-0.000,0.000,-0.000,0.000,-gistda


In [ ]:
temp_topic = topic_columns.copy()

In [ ]:
(dictionary)

array(['%...', '--', '-coffee', ..., '’ว', '’’', '….'], dtype=object)

In [ ]:
topic_encoded_df.head()

,topic_1,topic_2,topic_3,topic_4,topic_5,documents,sentence
0,0.019,-0.010,0.003,-0.009,0.005,ประกันสังคม ยะลา ผู้ประกันตน มาตรา 40 เงินสมทบ...,ประกันสังคมยะลา ขอผู้ประกันตนมาตรา 40 ที่ยังไม...
1,0.078,-0.047,-0.000,-0.057,0.033,หน่วยพัฒนาการเคลื่อนที่ 56 อุบลราชธานี ให้การ ...,นพค. 56 อุบลราชธานี ให้การช่วยเหลือ รื้อถอน ตั...
2,0.123,-0.068,0.003,-0.116,0.009,กาชาด ยะลา มอบ ถุง ยังชีพ สนับสนุน ขับเคลื่อน ...,กาชาดยะลา มอบถุงยังชีพสนับสนุนการขับเคลื่อนการ...
3,0.079,-0.051,0.010,0.044,0.022,กิจกรรม โครงการ ลด ความเหลื่อมล้ำ การศึกษา เทศ...,"กิจกรรม ""โครงการลดความเหลื่อมล้ำทางการศึกษา""เท..."
4,0.091,-0.046,0.011,-0.045,0.066,ประชาชน จังหวัด อำนาจเจริญ ลงทะเบียน บัตร สวัส...,ประชาชนจังหวัดอำนาจเจริญ สามารถลงทะเบียนบัตร?ส...


In [ ]:
qtopic = []
for st in topic_columns:
    temp_topic = topic_columns.copy()
    temp_topic.remove(st)
    temp_topic = [st] + temp_topic
    sub_query = ', '.join(temp_topic)
    sql = "select words, %s from encoding_matrix order by %s desc limit 10" % (sub_query, st)
    print(sql)
    #qtopic.append(pandasql.sqldf(sql))
    qtopic.append(duckdb.query(sql).to_df())

select words, topic_1, topic_2, topic_3, topic_4, topic_5 from encoding_matrix order by topic_1 desc limit 10
select words, topic_2, topic_1, topic_3, topic_4, topic_5 from encoding_matrix order by topic_2 desc limit 10
select words, topic_3, topic_1, topic_2, topic_4, topic_5 from encoding_matrix order by topic_3 desc limit 10
select words, topic_4, topic_1, topic_2, topic_3, topic_5 from encoding_matrix order by topic_4 desc limit 10
select words, topic_5, topic_1, topic_2, topic_3, topic_4 from encoding_matrix order by topic_5 desc limit 10


In [ ]:
topic_df = qtopic[0].head(10)
topic_df

,words,topic_1,topic_2,topic_3,topic_4,topic_5
0,จังหวัด,0.423,-0.104,0.032,-0.085,-0.108
1,2566,0.236,-0.027,-0.063,0.243,-0.208
2,เนื่องใน,0.170,0.227,-0.219,0.034,-0.105
3,พื้นที่,0.165,-0.158,0.051,-0.164,0.214
4,ประชุม,0.158,-0.142,0.031,-0.005,-0.182
5,พิธี,0.155,0.251,0.071,0.023,0.015
6,ตรัง,0.141,-0.110,0.021,-0.018,-0.141
7,โครงการ,0.140,-0.067,0.053,-0.124,0.037
8,ประจำปี,0.134,-0.009,-0.063,0.025,-0.140
9,พัฒนา,0.130,-0.037,-0.083,-0.120,-0.105


In [ ]:
len(qtopic)

5

In [ ]:
keyword_list = []

for i in range(len(qtopic)):
    topic_df = qtopic[i].head(10)

    keyword = []

    for i in range(topic_df.shape[0]):
        find_max = topic_df.iloc[ i , : ].to_list()
        max_index = np.argmax(find_max[1:])
        if max_index == 0 and find_max[1] > PREDICT_THRESHOLD:
            keyword.append(find_max[0])

    keyword_list.append(keyword)

In [ ]:
keyword_list

[['จังหวัด', 'ประชุม', 'ตรัง', 'โครงการ', 'ประจำปี', 'พัฒนา'],
 ['พิธี', 'เนื่องใน', 'สมเด็จ', 'ถวาย', 'บาท'],
 ['สมเด็จพระเจ้าลูกเธอ',
  'พัชรกิติยาภา',
  'เจ้าฟ้า',
  'เจริญ',
  'ถวายพระพร',
  'แด่',
  'มหาวัชรราชธิดา',
  'กรมหลวงราชสาริณีสิริพัชร',
  'นเรนทิราเทพยวดี',
  'ชัยมงคล'],
 ['อุบัติเหตุ',
  'ถนน',
  'เทศกาล',
  'ลด',
  '2566',
  'ปีใหม่',
  'สงกรานต์',
  'ป้องกัน',
  'ศูนย์ปฏิบัติการ',
  'จุด'],
 ['บรม',
  'พื้นที่',
  'ประชาชน',
  'อำเภอ',
  'ตรวจ',
  'บพิตร',
  'ธิเบศร',
  'มหา',
  'ชน',
  'นาถ']]

In [ ]:
topic_name = [st[0] for st in keyword_list] # ดึง keywords แรกของแต่ละ topic (1-4)
topic_name

['จังหวัด', 'พิธี', 'สมเด็จพระเจ้าลูกเธอ', 'อุบัติเหตุ', 'บรม']

In [ ]:
topic_df = pd.DataFrame(topic_name, columns=['topic_name'])
topic_df

,topic_name
0,จังหวัด
1,พิธี
2,สมเด็จพระเจ้าลูกเธอ
3,อุบัติเหตุ
4,บรม


In [ ]:
topic_name_list = [', '.join(map(str, st)) for st in keyword_list]
topic_name_list

['จังหวัด, ประชุม, ตรัง, โครงการ, ประจำปี, พัฒนา',
 'พิธี, เนื่องใน, สมเด็จ, ถวาย, บาท',
 'สมเด็จพระเจ้าลูกเธอ, พัชรกิติยาภา, เจ้าฟ้า, เจริญ, ถวายพระพร, แด่, มหาวัชรราชธิดา, กรมหลวงราชสาริณีสิริพัชร, นเรนทิราเทพยวดี, ชัยมงคล',
 'อุบัติเหตุ, ถนน, เทศกาล, ลด, 2566, ปีใหม่, สงกรานต์, ป้องกัน, ศูนย์ปฏิบัติการ, จุด',
 'บรม, พื้นที่, ประชาชน, อำเภอ, ตรวจ, บพิตร, ธิเบศร, มหา, ชน, นาถ']

In [ ]:
temp_df = pd.DataFrame(topic_name_list, columns=['topic_name_list'])

In [ ]:
topic_df = pd.concat([topic_df, temp_df], axis=1)
topic_df.set_index('topic_name')
topic_df

,topic_name,topic_name_list
0,จังหวัด,"จังหวัด, ประชุม, ตรัง, โครงการ, ประจำปี, พัฒนา"
1,พิธี,"พิธี, เนื่องใน, สมเด็จ, ถวาย, บาท"
2,สมเด็จพระเจ้าลูกเธอ,"สมเด็จพระเจ้าลูกเธอ, พัชรกิติยาภา, เจ้าฟ้า, เจ..."
3,อุบัติเหตุ,"อุบัติเหตุ, ถนน, เทศกาล, ลด, 2566, ปีใหม่, สงก..."
4,บรม,"บรม, พื้นที่, ประชาชน, อำเภอ, ตรวจ, บพิตร, ธิเ..."


In [ ]:
temp_topic_encoded_df = pd.DataFrame(lsa, columns = topic_name)
temp_topic_encoded_df["sentence"] = qdf['sentence']

In [ ]:
temp_topic_encoded_df

,จังหวัด,พิธี,สมเด็จพระเจ้าลูกเธอ,อุบัติเหตุ,บรม,sentence
0,0.019,-0.010,0.003,-0.009,0.005,ประกันสังคมยะลา ขอผู้ประกันตนมาตรา 40 ที่ยังไม...
1,0.078,-0.047,-0.000,-0.057,0.033,นพค. 56 อุบลราชธานี ให้การช่วยเหลือ รื้อถอน ตั...
2,0.123,-0.068,0.003,-0.116,0.009,กาชาดยะลา มอบถุงยังชีพสนับสนุนการขับเคลื่อนการ...
3,0.079,-0.051,0.010,0.044,0.022,"กิจกรรม ""โครงการลดความเหลื่อมล้ำทางการศึกษา""เท..."
4,0.091,-0.046,0.011,-0.045,0.066,ประชาชนจังหวัดอำนาจเจริญ สามารถลงทะเบียนบัตร?ส...
...,...,...,...,...,...,...
54774,0.070,-0.035,-0.004,-0.060,-0.007,บริษัท NT นำโครงการเพาะพันธุ์ดี NT Youth Club ...
54775,0.115,0.055,-0.008,-0.019,-0.077,ชมรมแม่บ้านมหาดไทยจังหวัดอำนาจเจริญ รถเข็นนั่ง...
54776,0.204,0.211,-0.079,-0.005,-0.136,เหล่ากาชาดอำนาจเจริญมอบบ้านให้ผู้ด้อยโอกาส ผู้...
54777,0.170,0.026,0.009,-0.075,-0.047,ชมรมแม่บ้านมหาดไทยจังหวัดอำนาจเจริญ มอบบ้านตาม...


In [ ]:
temp_df_list = []

for i in range(temp_topic_encoded_df.shape[0]):
    find_sub_dimension = temp_topic_encoded_df.iloc[ i , : ].to_list()
    max_index = np.argmax(find_sub_dimension[0:TOPIC_NUMBER])
    if find_sub_dimension[max_index] > 0:
        sub_dimension = topic_name[max_index]
    else:
        sub_dimension = 'n/a'

    temp = find_sub_dimension + [sub_dimension]
    temp_df_list.append(temp)

In [ ]:
temp_df = pd.DataFrame(temp_df_list, columns = topic_name + ['sentence', 'sub_dimension'])
temp_df.head()

,จังหวัด,พิธี,สมเด็จพระเจ้าลูกเธอ,อุบัติเหตุ,บรม,sentence,sub_dimension
0,0.019,-0.010,0.003,-0.009,0.005,ประกันสังคมยะลา ขอผู้ประกันตนมาตรา 40 ที่ยังไม...,จังหวัด
1,0.078,-0.047,-0.000,-0.057,0.033,นพค. 56 อุบลราชธานี ให้การช่วยเหลือ รื้อถอน ตั...,จังหวัด
2,0.123,-0.068,0.003,-0.116,0.009,กาชาดยะลา มอบถุงยังชีพสนับสนุนการขับเคลื่อนการ...,จังหวัด
3,0.079,-0.051,0.010,0.044,0.022,"กิจกรรม ""โครงการลดความเหลื่อมล้ำทางการศึกษา""เท...",จังหวัด
4,0.091,-0.046,0.011,-0.045,0.066,ประชาชนจังหวัดอำนาจเจริญ สามารถลงทะเบียนบัตร?ส...,จังหวัด


In [ ]:
temp_df.shape

(54779, 7)

In [ ]:
q_sentence = []

for st in topic_name:
    #qdf = pandasql.sqldf("select sentence as '%s' from temp_df where sub_dimension='%s'" % (st, st))
    sql = "select sentence as '%s' from temp_df where sub_dimension='%s'" % (st, st)
    qdf = duckdb.query(sql).to_df()

    q_sentence.append(qdf.head(5))

In [ ]:
sentence_df = pd.concat(q_sentence, axis=1)
sentence_df

,จังหวัด,พิธี,สมเด็จพระเจ้าลูกเธอ,อุบัติเหตุ,บรม
0,ประกันสังคมยะลา ขอผู้ประกันตนมาตรา 40 ที่ยังไม...,ผู้ว่าฯศรีสะเกษ เป็นประธานพิธีเจริญพระพุทธมนต์...,พอใจภาพรวมการแก้ไขปัญหาความยากจนแบบพุ่งเป้า ได...,จ.นครศรีธรรมราช ประชุมคณะกรรมการศูนย์อำนวยการค...,ยะลา ประชาชน ทยอย จับจ่าย “คนละครึ่งเฟส 5” วันแรก
1,นพค. 56 อุบลราชธานี ให้การช่วยเหลือ รื้อถอน ตั...,"มอบ ""รถเข็นนั่ง ปันสุข"" เพื่อเฉลิมพระเกียรติ ส...",กอ.รมน.ศรีสะเกษ เข้าร่วมพิธีเจริญพระพุทธมนต์ แ...,ขนส่งชัยภูมิ ตั้งจุดตรวจสอบรถและคนขับรถตามกฎหม...,กรมอนามัย แนะประชาชนดูแลสุขภาพตนเอง ลดเสี่ยงโร...
2,กาชาดยะลา มอบถุงยังชีพสนับสนุนการขับเคลื่อนการ...,นายอำเภอโพธิ์ศรีสุวรรณ เป็นประธานในพิธีบุญมหาส...,ตำรวจภูธรภาค 3 เตือนเล่นพนันทางเว็บไซต์ออนไลน์...,ปลัดจังหวัดยโสธร นำตรวจเยี่ยมและติดตามการป้องก...,กองสาธารณสุขและสิ่งแวดล้อม เทศบาลเมืองเบตงออกห...
3,"กิจกรรม ""โครงการลดความเหลื่อมล้ำทางการศึกษา""เท...","การแถลงข่าว ""พิธีวางศิลาฤกษ์ เททอง หล่อรูปเหมื...",จ.สุรินทร์ เตรียมความพร้อมด้านอาหารและน้ำดื่ม ...,เกษตรกรเลี้ยงจิ้งหรีดขอนแก่น ระบุอาหารสัตว์แพง...,นอภ.โพธิ์ศรีสุวรรณ นำส่วนราชการที่เกี่ยวข้อง ...
4,ประชาชนจังหวัดอำนาจเจริญ สามารถลงทะเบียนบัตร?ส...,จ.ประจวบฯ มอบกรรมสิทธิ์และไถ่ชีวิตโคเฉลิมพระเก...,ขอแสดงความยินดี แด่พนักงานครู สังกัดเทศบาลนครยะลา,ผู้อำนวยการแขวงการทางระยองเผย เส้นทางน้ำท่วมถน...,อำเภอขุนหาญ ออกให้บริการจัดทำบัตรประจำตัวประชา...


In [ ]:
sentence_df = sentence_df.transpose()
sentence_df = sentence_df.rename_axis('topic_name')
# sentence_df.columns.names = ['topic_name']
sentence_df

,0,1,2,3,4
topic_name,,,,,
จังหวัด,ประกันสังคมยะลา ขอผู้ประกันตนมาตรา 40 ที่ยังไม...,นพค. 56 อุบลราชธานี ให้การช่วยเหลือ รื้อถอน ตั...,กาชาดยะลา มอบถุงยังชีพสนับสนุนการขับเคลื่อนการ...,"กิจกรรม ""โครงการลดความเหลื่อมล้ำทางการศึกษา""เท...",ประชาชนจังหวัดอำนาจเจริญ สามารถลงทะเบียนบัตร?ส...
พิธี,ผู้ว่าฯศรีสะเกษ เป็นประธานพิธีเจริญพระพุทธมนต์...,"มอบ ""รถเข็นนั่ง ปันสุข"" เพื่อเฉลิมพระเกียรติ ส...",นายอำเภอโพธิ์ศรีสุวรรณ เป็นประธานในพิธีบุญมหาส...,"การแถลงข่าว ""พิธีวางศิลาฤกษ์ เททอง หล่อรูปเหมื...",จ.ประจวบฯ มอบกรรมสิทธิ์และไถ่ชีวิตโคเฉลิมพระเก...
สมเด็จพระเจ้าลูกเธอ,พอใจภาพรวมการแก้ไขปัญหาความยากจนแบบพุ่งเป้า ได...,กอ.รมน.ศรีสะเกษ เข้าร่วมพิธีเจริญพระพุทธมนต์ แ...,ตำรวจภูธรภาค 3 เตือนเล่นพนันทางเว็บไซต์ออนไลน์...,จ.สุรินทร์ เตรียมความพร้อมด้านอาหารและน้ำดื่ม ...,ขอแสดงความยินดี แด่พนักงานครู สังกัดเทศบาลนครยะลา
อุบัติเหตุ,จ.นครศรีธรรมราช ประชุมคณะกรรมการศูนย์อำนวยการค...,ขนส่งชัยภูมิ ตั้งจุดตรวจสอบรถและคนขับรถตามกฎหม...,ปลัดจังหวัดยโสธร นำตรวจเยี่ยมและติดตามการป้องก...,เกษตรกรเลี้ยงจิ้งหรีดขอนแก่น ระบุอาหารสัตว์แพง...,ผู้อำนวยการแขวงการทางระยองเผย เส้นทางน้ำท่วมถน...
บรม,ยะลา ประชาชน ทยอย จับจ่าย “คนละครึ่งเฟส 5” วันแรก,กรมอนามัย แนะประชาชนดูแลสุขภาพตนเอง ลดเสี่ยงโร...,กองสาธารณสุขและสิ่งแวดล้อม เทศบาลเมืองเบตงออกห...,นอภ.โพธิ์ศรีสุวรรณ นำส่วนราชการที่เกี่ยวข้อง ...,อำเภอขุนหาญ ออกให้บริการจัดทำบัตรประจำตัวประชา...


In [ ]:
sentence_df.index

Index(['จังหวัด', 'พิธี', 'สมเด็จพระเจ้าลูกเธอ', 'อุบัติเหตุ', 'บรม'], dtype='object', name='topic_name')

In [ ]:
sentence_df['topic_name'] = sentence_df.index

In [ ]:
sentence_df.head()

,0,1,2,3,4,topic_name
topic_name,,,,,,
จังหวัด,ประกันสังคมยะลา ขอผู้ประกันตนมาตรา 40 ที่ยังไม...,นพค. 56 อุบลราชธานี ให้การช่วยเหลือ รื้อถอน ตั...,กาชาดยะลา มอบถุงยังชีพสนับสนุนการขับเคลื่อนการ...,"กิจกรรม ""โครงการลดความเหลื่อมล้ำทางการศึกษา""เท...",ประชาชนจังหวัดอำนาจเจริญ สามารถลงทะเบียนบัตร?ส...,จังหวัด
พิธี,ผู้ว่าฯศรีสะเกษ เป็นประธานพิธีเจริญพระพุทธมนต์...,"มอบ ""รถเข็นนั่ง ปันสุข"" เพื่อเฉลิมพระเกียรติ ส...",นายอำเภอโพธิ์ศรีสุวรรณ เป็นประธานในพิธีบุญมหาส...,"การแถลงข่าว ""พิธีวางศิลาฤกษ์ เททอง หล่อรูปเหมื...",จ.ประจวบฯ มอบกรรมสิทธิ์และไถ่ชีวิตโคเฉลิมพระเก...,พิธี
สมเด็จพระเจ้าลูกเธอ,พอใจภาพรวมการแก้ไขปัญหาความยากจนแบบพุ่งเป้า ได...,กอ.รมน.ศรีสะเกษ เข้าร่วมพิธีเจริญพระพุทธมนต์ แ...,ตำรวจภูธรภาค 3 เตือนเล่นพนันทางเว็บไซต์ออนไลน์...,จ.สุรินทร์ เตรียมความพร้อมด้านอาหารและน้ำดื่ม ...,ขอแสดงความยินดี แด่พนักงานครู สังกัดเทศบาลนครยะลา,สมเด็จพระเจ้าลูกเธอ
อุบัติเหตุ,จ.นครศรีธรรมราช ประชุมคณะกรรมการศูนย์อำนวยการค...,ขนส่งชัยภูมิ ตั้งจุดตรวจสอบรถและคนขับรถตามกฎหม...,ปลัดจังหวัดยโสธร นำตรวจเยี่ยมและติดตามการป้องก...,เกษตรกรเลี้ยงจิ้งหรีดขอนแก่น ระบุอาหารสัตว์แพง...,ผู้อำนวยการแขวงการทางระยองเผย เส้นทางน้ำท่วมถน...,อุบัติเหตุ
บรม,ยะลา ประชาชน ทยอย จับจ่าย “คนละครึ่งเฟส 5” วันแรก,กรมอนามัย แนะประชาชนดูแลสุขภาพตนเอง ลดเสี่ยงโร...,กองสาธารณสุขและสิ่งแวดล้อม เทศบาลเมืองเบตงออกห...,นอภ.โพธิ์ศรีสุวรรณ นำส่วนราชการที่เกี่ยวข้อง ...,อำเภอขุนหาญ ออกให้บริการจัดทำบัตรประจำตัวประชา...,บรม


In [ ]:
topic_df.topic_name_list

0       จังหวัด, ประชุม, ตรัง, โครงการ, ประจำปี, พัฒนา
1                    พิธี, เนื่องใน, สมเด็จ, ถวาย, บาท
2    สมเด็จพระเจ้าลูกเธอ, พัชรกิติยาภา, เจ้าฟ้า, เจ...
3    อุบัติเหตุ, ถนน, เทศกาล, ลด, 2566, ปีใหม่, สงก...
4    บรม, พื้นที่, ประชาชน, อำเภอ, ตรวจ, บพิตร, ธิเ...
Name: topic_name_list, dtype: object

# Result

In [ ]:
sql = 'select sentence_df.*, topic_df.topic_name_list  from topic_df inner join sentence_df on topic_df.topic_name = sentence_df.topic_name'

#df = pandasql.sqldf(sql)
sen_df = duckdb.query(sql).to_df()

In [ ]:
sen_df.head()

,0,1,2,3,4,topic_name,topic_name_list
0,ประกันสังคมยะลา ขอผู้ประกันตนมาตรา 40 ที่ยังไม...,นพค. 56 อุบลราชธานี ให้การช่วยเหลือ รื้อถอน ตั...,กาชาดยะลา มอบถุงยังชีพสนับสนุนการขับเคลื่อนการ...,"กิจกรรม ""โครงการลดความเหลื่อมล้ำทางการศึกษา""เท...",ประชาชนจังหวัดอำนาจเจริญ สามารถลงทะเบียนบัตร?ส...,จังหวัด,"จังหวัด, ประชุม, ตรัง, โครงการ, ประจำปี, พัฒนา"
1,ผู้ว่าฯศรีสะเกษ เป็นประธานพิธีเจริญพระพุทธมนต์...,"มอบ ""รถเข็นนั่ง ปันสุข"" เพื่อเฉลิมพระเกียรติ ส...",นายอำเภอโพธิ์ศรีสุวรรณ เป็นประธานในพิธีบุญมหาส...,"การแถลงข่าว ""พิธีวางศิลาฤกษ์ เททอง หล่อรูปเหมื...",จ.ประจวบฯ มอบกรรมสิทธิ์และไถ่ชีวิตโคเฉลิมพระเก...,พิธี,"พิธี, เนื่องใน, สมเด็จ, ถวาย, บาท"
2,พอใจภาพรวมการแก้ไขปัญหาความยากจนแบบพุ่งเป้า ได...,กอ.รมน.ศรีสะเกษ เข้าร่วมพิธีเจริญพระพุทธมนต์ แ...,ตำรวจภูธรภาค 3 เตือนเล่นพนันทางเว็บไซต์ออนไลน์...,จ.สุรินทร์ เตรียมความพร้อมด้านอาหารและน้ำดื่ม ...,ขอแสดงความยินดี แด่พนักงานครู สังกัดเทศบาลนครยะลา,สมเด็จพระเจ้าลูกเธอ,"สมเด็จพระเจ้าลูกเธอ, พัชรกิติยาภา, เจ้าฟ้า, เจ..."
3,จ.นครศรีธรรมราช ประชุมคณะกรรมการศูนย์อำนวยการค...,ขนส่งชัยภูมิ ตั้งจุดตรวจสอบรถและคนขับรถตามกฎหม...,ปลัดจังหวัดยโสธร นำตรวจเยี่ยมและติดตามการป้องก...,เกษตรกรเลี้ยงจิ้งหรีดขอนแก่น ระบุอาหารสัตว์แพง...,ผู้อำนวยการแขวงการทางระยองเผย เส้นทางน้ำท่วมถน...,อุบัติเหตุ,"อุบัติเหตุ, ถนน, เทศกาล, ลด, 2566, ปีใหม่, สงก..."
4,ยะลา ประชาชน ทยอย จับจ่าย “คนละครึ่งเฟส 5” วันแรก,กรมอนามัย แนะประชาชนดูแลสุขภาพตนเอง ลดเสี่ยงโร...,กองสาธารณสุขและสิ่งแวดล้อม เทศบาลเมืองเบตงออกห...,นอภ.โพธิ์ศรีสุวรรณ นำส่วนราชการที่เกี่ยวข้อง ...,อำเภอขุนหาญ ออกให้บริการจัดทำบัตรประจำตัวประชา...,บรม,"บรม, พื้นที่, ประชาชน, อำเภอ, ตรวจ, บพิตร, ธิเ..."


In [ ]:
colume_name = sen_df.columns.to_list()
colume_name

['0', '1', '2', '3', '4', 'topic_name', 'topic_name_list']

In [ ]:
colume_name.remove('topic_name_list')

In [ ]:
colume_name

['0', '1', '2', '3', '4', 'topic_name']

In [ ]:
colume_name.insert(1, 'topic_name_list')

In [ ]:
colume_name

['0', 'topic_name_list', '1', '2', '3', '4', 'topic_name']

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
sen_df.to_csv(TOPIC_NAME, encoding="utf8", columns = colume_name)

In [ ]:
sen_df = pd.read_csv(TOPIC_NAME, index_col=0)
sheet = from_dataframe(sen_df)
sheet

Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format=None, row_end=4, row_start=0, squeeze_row=False…

In [ ]:
sen_df = to_dataframe(sheet)
sen_df.to_csv(TOPIC_NAME, encoding="utf8", columns = colume_name)

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [ ]:
# df = pd.read_csv(TOPIC_NAME, index_col=0)
topic_name = sen_df['topic_name'].to_list()

In [ ]:
temp_df_list = []

for i in range(temp_topic_encoded_df.shape[0]):
    find_sub_dimension = temp_topic_encoded_df.iloc[ i , : ].to_list()
    max_index = np.argmax(find_sub_dimension[0:TOPIC_NUMBER])
    if find_sub_dimension[max_index] > PREDICT_THRESHOLD:
        sub_dimension = topic_name[max_index]
    else:
        sub_dimension = 'n/a'

    temp = find_sub_dimension + [sub_dimension]
    temp_df_list.append(temp)

In [ ]:
temp_df = pd.DataFrame(temp_df_list, columns = topic_name + ['sentence', 'sub_dimension'])
temp_df.head()

,จังหวัด,พิธี,สมเด็จพระเจ้าลูกเธอ,อุบัติเหตุ,บรม,sentence,sub_dimension
0,0.019,-0.010,0.003,-0.009,0.005,ประกันสังคมยะลา ขอผู้ประกันตนมาตรา 40 ที่ยังไม...,จังหวัด
1,0.078,-0.047,-0.000,-0.057,0.033,นพค. 56 อุบลราชธานี ให้การช่วยเหลือ รื้อถอน ตั...,จังหวัด
2,0.123,-0.068,0.003,-0.116,0.009,กาชาดยะลา มอบถุงยังชีพสนับสนุนการขับเคลื่อนการ...,จังหวัด
3,0.079,-0.051,0.010,0.044,0.022,"กิจกรรม ""โครงการลดความเหลื่อมล้ำทางการศึกษา""เท...",จังหวัด
4,0.091,-0.046,0.011,-0.045,0.066,ประชาชนจังหวัดอำนาจเจริญ สามารถลงทะเบียนบัตร?ส...,จังหวัด


In [ ]:
temp_df.sample(10)

,จังหวัด,พิธี,สมเด็จพระเจ้าลูกเธอ,อุบัติเหตุ,บรม,sentence,sub_dimension
32663,0.223,-0.095,-0.000,-0.111,0.016,สำนักงานสลากกินแบ่งรัฐบาล ร่วมกับ จังหวัดสตูลโ...,จังหวัด
4539,0.142,-0.091,0.031,-0.097,0.106,จังหวัดชัยนาท น้ำล้นตลิ่งเข้าท่วมพื้นที่อำเภอส...,จังหวัด
19223,0.067,-0.017,-0.003,-0.035,-0.008,"โครงการ ""คนปราจีนบุรี ร่วมสืบสานอนุรักษ์ศิลป์ผ...",จังหวัด
21492,0.175,0.300,0.312,0.017,0.044,จังหวัดนนทบุรี จัดพิธีทำบุญตักบาตรและเจริญพระพ...,สมเด็จพระเจ้าลูกเธอ
39531,0.082,-0.051,0.012,-0.033,0.025,สำนักงานสาธารณสุขจังหวัดบุรีรัมย์ ขับเคลื่อนระ...,จังหวัด
54303,0.008,-0.005,0.000,-0.004,-0.001,ผลักดันเชียงใหม่เป็นนครที่สุด แห่งความสง่างามท...,จังหวัด
27739,0.159,-0.113,0.035,-0.071,0.018,นายกรัฐมนตรี นำคณะลงพื้นที่ตรวจราชการจังหวัดอุ...,จังหวัด
25444,0.159,-0.082,0.012,0.014,-0.012,รองผู้ว่าฯ ปทุมธานี เปิดโครงการฝึกอบรมเยาวชนขั...,จังหวัด
474,0.047,-0.007,0.006,-0.008,0.002,ผู้ว่าฯ ยโสธร นำนายอำเภอและพุทธศาสนิกชน ทอดผ้า...,จังหวัด
26675,0.021,0.003,0.026,-0.007,0.007,วิทยาลัยอาชีศึกษาภูเก็ต จัดงาน “วันเกียรติยศ อ...,สมเด็จพระเจ้าลูกเธอ


In [ ]:
temp_df.to_csv(TOPIC, encoding='utf8', columns = ['sentence', 'sub_dimension'] + topic_name)